# The Philippine president's SONA

This scrapes the contents of https://www.officialgazette.gov.ph/past-sona-speeches/ for copies of the State of the Nation Addresses of Philippine presidents from 1936 to present.

The goal is to be able to use the SONAs for text analysis. Speeches are delivered before congress every fourth Monday of July and widely anticipated for setting the tone of a sitting administration.

An analysis is provided at the latter part of the notebook.

## Do your imports

In [31]:
import pandas as pd

import re
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

import requests
from bs4 import BeautifulSoup
import altair as alt

## Allow Selenium to open up Chrome and automatically navigate through the website

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the any command: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --version
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Driver [/Users/prinzmagtulis/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache
/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_18219/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
driver.get("https://www.officialgazette.gov.ph/past-sona-speeches/")

## Scraping proper: table

First step is to scrape all tabled information, that is, excluding all the contents of **links**.

In [4]:
rows= driver.find_elements(By.TAG_NAME, "tr")

We arrange the information into a **list of dictionaries** in preparation to transforming it into a **data frame** for pandas analysis later.

In [5]:
dataset=[]
for dicts in rows[1:]:
    data={}
    all_tds = dicts.find_elements(By.TAG_NAME, "td")
    if len(all_tds) == 5:
        prexy = data['president']= dicts.find_elements(By.TAG_NAME, "td")[0].text
        data['date']= dicts.find_elements(By.TAG_NAME, "td")[1].text
        data['title'] = dicts.find_elements(By.TAG_NAME, "td")[2].text
        try:
            data['link'] = dicts.find_elements(By.TAG_NAME, "a")[1].get_attribute('href')
        except:
            data['link'] = dicts.find_element(By.TAG_NAME, "a").get_attribute('href')
        data['venue'] = dicts.find_elements(By.TAG_NAME, "td")[3].text
        data['session'] = dicts.find_elements(By.TAG_NAME, "td")[4].text
        dataset.append(data)
    else:
        data['president'] = prexy
        data['date']= dicts.find_elements(By.TAG_NAME, "td")[0].text
        data['title'] = dicts.find_elements(By.TAG_NAME, "td")[1].text
        data['link'] = dicts.find_element(By.TAG_NAME, "a").get_attribute('href')
        data['venue'] = dicts.find_elements(By.TAG_NAME, "td")[2].text
        data['session'] = dicts.find_elements(By.TAG_NAME, "td")[3].text
        dataset.append(data)
dataset

[{'president': 'Manuel L. Quezon',
  'date': 'November 25, 1935',
  'title': 'Message to the First Assembly on National Defense',
  'link': 'http://www.officialgazette.gov.ph/1935/11/25/message-of-president-quezon-to-the-first-assembly-on-national-defense-november-25-1935/',
  'venue': 'Legislative Building, Manila',
  'session': 'First National Assembly, First Session'},
 {'president': 'Manuel L. Quezon',
  'date': 'June 16, 1936',
  'title': 'On the Country’s Conditions and Problems',
  'link': 'http://www.officialgazette.gov.ph/1936/06/16/manuel-l-quezon-second-state-of-the-nation-address-june-16-1936/',
  'venue': 'Legislative Building, Manila',
  'session': 'First National Assembly, First Session'},
 {'president': 'Manuel L. Quezon',
  'date': 'October 18, 1937',
  'title': 'Improvement of Philippine Conditions, Philippine Independence, and Relations with American High Commissioner',
  'link': 'http://www.officialgazette.gov.ph/1937/10/18/manuel-l-quezon-third-state-of-the-nation-

Our **first data frame**

In [6]:
df1 = pd.DataFrame(dataset)
df1.head()

,president,date,title,link,venue,session
0,Manuel L. Quezon,"November 25, 1935",Message to the First Assembly on National Defense,http://www.officialgazette.gov.ph/1935/11/25/m...,"Legislative Building, Manila","First National Assembly, First Session"
1,Manuel L. Quezon,"June 16, 1936",On the Country’s Conditions and Problems,http://www.officialgazette.gov.ph/1936/06/16/m...,"Legislative Building, Manila","First National Assembly, First Session"
2,Manuel L. Quezon,"October 18, 1937","Improvement of Philippine Conditions, Philippi...",http://www.officialgazette.gov.ph/1937/10/18/m...,"Legislative Building, Manila","First National Assembly, Second Session"
3,Manuel L. Quezon,"January 24, 1938",Revision of the System of Taxation,http://www.officialgazette.gov.ph/1938/01/24/m...,"Legislative Building, Manila","First National Assembly, Third Session"
4,Manuel L. Quezon,"January 24, 1939",The State of the Nation and Important Economic...,http://www.officialgazette.gov.ph/1939/01/24/m...,"Legislative Building, Manila","Second National Assembly, First Session"


## Scraping proper: actual speeches

We use BeautifulSoup on this one. The process is easier since we already have the links in the first df and all we have to do is to just **access and grab** their contents one by one.

I'm commenting this part out to avoid reading through a bunch of texts, but hey, it runs very well so try it on your own!

In [7]:
speeches=[]
for speech in dataset[0:]:
    href = speech['link']
    raw_html = requests.get(href).content
    doc = BeautifulSoup(raw_html, "html.parser")
    headers = doc.find_all(class_= 'large-9 large-centered columns')[1]
    text={}
    text['link']= speech['link']
    text['speech']= headers.text 
    speeches.append(text)
#speeches

As you can see, the speeches are arranged as a **single block** per row to match their place in the df. This is, of course, not the ideal way and may be improved. Below is a **second data frame** containing the links and speeches themselves.

We then **merge** this information with our earlier df.

In [8]:
df2=pd.DataFrame(speeches)
df2

,link,speech
0,http://www.officialgazette.gov.ph/1935/11/25/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
1,http://www.officialgazette.gov.ph/1936/06/16/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
2,http://www.officialgazette.gov.ph/1937/10/18/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
3,http://www.officialgazette.gov.ph/1938/01/24/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
4,http://www.officialgazette.gov.ph/1939/01/24/m...,\nMessage\nof\nHis Excellency Manuel L. Quezon...
...,...,...
79,https://www.officialgazette.gov.ph/2018/07/23/...,\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
80,https://www.officialgazette.gov.ph/2019/07/22/...,\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
81,https://www.officialgazette.gov.ph/2020/07/27/...,\n\n\n\n\n\n\n5TH STATE OF THE NATION ADDRESS ...
82,https://www.officialgazette.gov.ph/2021/07/26/...,\n\n\tState of the Nation Address of \n\tRodri...


Our final df.

In [9]:
merged = df1.merge(df2, suffixes=('_left'))
merged

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_18219/4263006727.py:1: FutureWarning: Passing 'suffixes' as a <class 'str'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  merged = df1.merge(df2, suffixes=('_left'))


,president,date,title,link,venue,session,speech
0,Manuel L. Quezon,"November 25, 1935",Message to the First Assembly on National Defense,http://www.officialgazette.gov.ph/1935/11/25/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
1,Manuel L. Quezon,"June 16, 1936",On the Country’s Conditions and Problems,http://www.officialgazette.gov.ph/1936/06/16/m...,"Legislative Building, Manila","First National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
2,Manuel L. Quezon,"October 18, 1937","Improvement of Philippine Conditions, Philippi...",http://www.officialgazette.gov.ph/1937/10/18/m...,"Legislative Building, Manila","First National Assembly, Second Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
3,Manuel L. Quezon,"January 24, 1938",Revision of the System of Taxation,http://www.officialgazette.gov.ph/1938/01/24/m...,"Legislative Building, Manila","First National Assembly, Third Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
4,Manuel L. Quezon,"January 24, 1939",The State of the Nation and Important Economic...,http://www.officialgazette.gov.ph/1939/01/24/m...,"Legislative Building, Manila","Second National Assembly, First Session",\nMessage\nof\nHis Excellency Manuel L. Quezon...
...,...,...,...,...,...,...,...
79,Rodrigo Roa Duterte,"July 23, 2018",Third State of the Nation Address,https://www.officialgazette.gov.ph/2018/07/23/...,"Batasang Pambansa, Quezon City","Seventeenth Congress, Third Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
80,Rodrigo Roa Duterte,"July 22, 2019",Fourth State of the Nation Address,https://www.officialgazette.gov.ph/2019/07/22/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, First Session",\n\n\n\nSTATE OF THE NATION ADDRESS OF \nRODRI...
81,Rodrigo Roa Duterte,"July 27, 2020",Fifth State of the Nation Address,https://www.officialgazette.gov.ph/2020/07/27/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Second Session",\n\n\n\n\n\n\n5TH STATE OF THE NATION ADDRESS ...
82,Rodrigo Roa Duterte,"July 26, 2021",Sixth State of the Nation Address,https://www.officialgazette.gov.ph/2021/07/26/...,"Batasang Pambansa, Quezon City","Eighteenth Congress, Third Session",\n\n\tState of the Nation Address of \n\tRodri...


## Save to CSV

In [10]:
#merged.to_csv('merged.csv', index=False)

# Initial analysis

## regex

We are now ready to take an **initial analysis** of the texts that we have. For this part, I provided some examples below using **regex**.

An important note on this method: the **str.contains** and **str.extractall** functions **ONLY** count *the number of speeches* that contain the word, *not how many times* the word was mentioned in the speech. We would look into the count of the words on the speeches later at a deeper analysis.

Words we ran here are based from peer-reviewed textual studies that gauge **populism.**

In [11]:
#Ran to just check the type of files we are dealing with.
merged.dtypes

president    object
date         object
title        object
link         object
venue        object
session      object
speech       object
dtype: object

### 'elite'

The word "elite" is found to have been often used by populist leaders. We find based on this initial analysis that in the case of Philippine presidents, three leaders (one of whom was **dictator** Ferdinand Marcos Sr.) were found to have included the word in their SONAs.

In [12]:
merged[merged.speech.str.contains(r"\belite", case=False, regex=True)].president.value_counts()

Ferdinand E. Marcos        2
Joseph Ejercito Estrada    1
Rodrigo Roa Duterte        1
Name: president, dtype: int64

In [13]:
pd.set_option('display.max_colwidth', None)
merged.speech.str.extractall(r'(.*\belite.+)', re.IGNORECASE)

,,0
,match,
31,0,"It is fortunate that the nation will, just two years from now, call a constitutional convention. I leave it to the delegates of that convention to evolve a truly democratic system, one which will not merely bend, as our system does today, to the wishes of a traditional elite and perpetuate the status quo. Democratic institutions must be instruments of national advancement. Democracy must symbolize change."
37,0,"Clearly, we face here the danger that our New Society is giving birth to a new government elite, who resurrect in our midst the privileges we fought in the past, who employ the powers of high office for their personal enrichment, as well as of their business colleagues, relatives, and friends."
60,0,"Our war on poverty is in the acceleration of the land redistribution processes under the agrarian reform program. We distributed more than 266,000 hectares of land to 175,000 landless farmers, including land owned by the traditional rural elite. [Applause]"
81,0,Great wealth enables economic elites and corporations to influence public policy to their advantage. Media is a powerful tool in the hands of oligarchs like the Lopezes who used their media outlets in their battles with political figures. I am a casualty of the Lopezes during the 2016 election.


### 'democracy' and 'demokrasya'

Dictator Ferdinand E. Marcos mentioned the word **"democracy"** in 10 of his SONAs followed by Gloria Arroyo (7 of 9 SONAs). In Filipino, Benigno Aquino III mentioned **"demokrasya"** in two of his six speeches. 



**Joseph Estrada**, whose term was cut short by a popular revolt in 2001, and **Rodrigo Duterte** mentioned the word in a single SONA. 

In [14]:
merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_18219/3410026565.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemocracy.+)", case=False, regex=True)].president.value_counts()


Ferdinand E. Marcos        10
Gloria Macapagal-Arroyo     7
Manuel L. Quezon            5
Corazon C. Aquino           5
Fidel V. Ramos              5
Ramon Magsaysay             4
Diosdado Macapagal          4
Manuel Roxas                3
Elpidio Quirino             3
Carlos P. Garcia            2
Joseph Ejercito Estrada     1
Rodrigo Roa Duterte         1
Name: president, dtype: int64

In [15]:
merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()

/var/folders/01/dz49lpcd4qq_yksvf114xzfc0000gn/T/ipykernel_18219/1026723483.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  merged[merged.speech.str.contains(r"(.*\bdemokrasya.+)", case=False, regex=True)].president.value_counts()


Benigno S. Aquino III      2
Ferdinand E. Marcos        1
Corazon C. Aquino          1
Gloria Macapagal-Arroyo    1
Name: president, dtype: int64

In [16]:
merged.speech.str.extractall(r'(.*\bdemocracy.+)', re.IGNORECASE).head(7)

0
  match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
1 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        In our day and generation democracy, as an effective system of government, is being challenged. Let this new democracy of ours show to the world that democracy can be as efficient as a dictatorship, without trespassing upon individual liberty and the sacred rights of the people.
2 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      Still more: The Filipino workingman has heard, if he is not able to read, of the equality before the law of the poor and the rich. He has heard of democracy, liberty, and justice, since every candidate for an elective office discourses on these topics, painting to him in glowing terms the meaning of these words.
  1                                                                                                                                                                                                                                                                                                             One of the discoveries which we have made since the establishment of the Government of the Commonwealth is that, despite the large number of children that have gone through our public schools, as shown in the reports of the Bureau of Education, the literacy of the Islands has not increased proportionally, and the knowledge of those rudimentary subjects which the citizen of a democracy should have, has not been acquired by a population corresponding to the number of children that appear to have entered the public schools. The reason for this is simple. A large proprtion of the boys and girls who have been admitted to the schools have not remained long en

In [17]:
merged.speech.str.extractall(r'(.*\bdemokrasya.+)', re.IGNORECASE).head()

0
   match                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
40 0                            Nasa harap ng kapulungang ito ngayon ang katipunan ng mga hamon at pagsubok sa nakalipas na mga Kongreso, at ito na sana ang pangwakas na pagsubok kung makakaya natin gamitin ang demokrasya bilang mabisang sangkap ng katatagan at kaunlarang pambansa. Bagaman at kailangan pa ring magpatuloy ang pansamantalang pamahalaan, taglay ng kapulungang ito ang binhi ng matatag at masiglang lehislaturang tutugon sa ating pangangailangan kung ihahandog natin dito ang lahat ng ating talino at kakayahan.
   1      Tayo ngayon ay isang bansang pinalakas ng mga pagsubok na ating pinagdaanan, higit na nagkakaisa pagkaraan ng mga sigalutang dinanas, at higit na handa sa anumang uri ng pagsubok at suliranin. Natapos nating lampasan ang mahihigpit na balakid sa nakaraang lima-at-kalahating taon. Sa liwanag ng makabuluhang yugtong ito ng ating buhay bilang bansa at lahi, magagawa natin ang ating tungkuling pagtahak sa landas ng katuparan ng ating matayog na pangarap na pag-unlad, pagkakapantay-pantay, at ng tunay na demokrasya.
51 0                                                                                                                                                                                                                                                                                                                                                                                                Binigyang buhay ng mga Kabisig nating ito ang diwa ng ating Saligang Batas; binigyang halimbawa nila ang tunay na kahulugan ng demokrasya.
   1                                                                                                                                                                                May katiyakan ang ating tagumpay kung tayo’y magkakaisa. Kung kaya’t hinihimok ko kayo—kagalang-galang na mga Senador, Kongresista, at ang iba pang mga pinuno ng bayan—na muli tayong manumpa sa pangarap na nagbigkis sa atin noong 1986: ibalik at panatiliin ang demokrasya, kalayaan, karapatan, katatagang pangkabuhayan, at katarungang panlipunan.
65 0                                                                                                                                                                                                                                            Pinapangako ko ang isang bagong direksyon: mamamayan muna. Ang taong bayan ang pinakamalaki nating yaman. Ngunit madalas, kaunti lang ang atensyon na binibigay sa kanilang pag-unlad. Di tuloy matawid ang agwat ng mayaman at mahirap. Di tuloy mapa-abot sa lahat ang biyaya ng demokrasya.

## Segregating by president

We create separate dataframes from a select number of presidents to analyze using text analysis.

In [80]:
#Post-martial law
cory = merged[(merged['president'] == 'Corazon C. Aquino')] #Cory Aquino
ramos = merged[(merged['president'] == 'Fidel V. Ramos')] #Fidel Ramos
aquino = merged[(merged['president'] == 'Benigno S. Aquino III')] #Aquino
duterte = merged[(merged['president'] == 'Rodrigo Roa Duterte')] #Duterte
erap = merged[(merged['president'] == 'Joseph Ejercito Estrada')] #Erap
arroyo = merged[(merged['president'] == 'Gloria Macapagal-Arroyo')] #Arroyo
marcosjr = merged[(merged['president'] == 'Ferdinand R. Marcos Jr.')] #Marcos Jr.

marcos = merged[(merged['president'] == 'Ferdinand E. Marcos')] #Marcos Sr.

# Pre-martial law
macapagal = merged[(merged['president'] == 'Diosdado Macapagal')] #Diosdado Macapagal
garcia = merged[(merged['president'] == 'Carlos P. Garcia')] #Carlos Garcia
magsaysay = merged[(merged['president'] == 'Ramon Magsaysay')] #Ramon Magsaysay
quirino = merged[(merged['president'] == 'Elpidio Quirino')] #Elpidio Quirino

## Import text analysis libraries and identify paramaters

We will use Python's National Language Toolkit and [Scikit-Learn](https://scikit-learn.org/stable/index.html).

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import stopwordsiso as stopwords

In [20]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    return text #removes all numbers

In [21]:
y_columns = ['president', 'speeches']
BINARY=False
NGRAM_RANGE=(1,1)
MIN_DF=0
STPWORDS=stopwords.stopwords(["en", "tl"]) #removes Tagalog stopwords
STPWORDS.update(['yung', 'iyan', 'yan', 'diyan', 'applause', 'laughter', 'palakpakan', 'rin', 'din', 'po',
                'pong', 'pang', 'pa', 'nang', 'ng', 'pag',
                'kapag', 'nga']) #adds more Tagalog stopwords not included in the package 

vectorizer = CountVectorizer(
    stop_words=STPWORDS,
    ngram_range=NGRAM_RANGE,
    binary=BINARY,
    min_df=MIN_DF,
    preprocessor=preprocess_text
)

## Vectorizing

Simple counting of words that occur in a speech. We use the latest SONA here of the **current president Ferdinand Marcos Jr.** as an example.

In [22]:
X = vectorizer.fit_transform(marcosjr['speech'])
X

/Users/prinzmagtulis/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


<1x1956 sparse matrix of type '<class 'numpy.int64'>'
	with 1956 stored elements in Compressed Sparse Row format>

In [23]:
marcos_vectors = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# [print(x) for x in marcosjr.speech]
marcos_vectors.round(2)

,aagapay,aalaga,aalis,aaral,aatasan,abandon,ability,abolition,abono,abreast,...,wind,windmill,wireless,women,workers,workplace,worse,yields,zimmerman,zubiri
0,1,1,1,1,1,1,1,1,1,1,...,3,1,1,1,6,1,1,1,1,1


## Simple word count

In [24]:
pd.set_option('display.max_columns', None)
marcos_vectors = marcos_vectors.transpose()

In [25]:
marcos_vectors.columns = ['SONA1']
marcos_vectors.sort_values('SONA1', ascending=False).head(20)

,SONA1
natin,32
government,25
department,22
national,22
philippines,21
power,18
development,18
energy,17
upang,16
reform,15


## Marcos Jr. first SONA

In [71]:
X = vectorizer.fit_transform(marcosjr['speech'])
marcos_idf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#[print(x) for x in speeches.sentence]
marcos_idf.round(2)

,aagapay,aalaga,aalis,aaral,aatasan,abandon,ability,abolition,abono,abreast,abundance,academic,accelerated,accelerating,access,accessible,accomplish,accountability,accountable,accurate,acquired,acquisition,activities,activity,actual,adapt,address,addressed,addressing,adjusted,adjustments,administrasyon,administration,admission,adopt,adoption,adopts,advance,advancement,advantage,advertising,affairs,affordability,aforementioned,afternoon,agad,agarang,age,agencies,agency,agenda,aggressive,agile,agrarian,agree,agricultural,agriculture,agrikultura,agrikultural,ahensiya,ahensya,aics,aid,aim,aims,airport,airports,akma,akong,alagaan,alam,alert,alerts,alexander,alleged,alleviate,alliances,allied,allocation,ama,ambassadors,ambiguities,amend,amended,amending,amendments,amortization,anak,anchor,annual,annually,anong,antiquated,apatnapung,apostolic,appropriation,approved,arabia,aralan,araneta,araw,archives,arena,arisen,arising,armed,arroyo,artificial,arts,aspire,assembled,assist,assistance,assumptions,atomic,attached,attain,attainment,attract,attractive,augmented,authority,automate,availability,avenues,average,aversion,awarded,ayaw,ayuda,babagay,babuyan,backbone,bagong,bahagi,bakuna,balance,balansehin,band,banda,bank,bansa,banta,barko,barrel,based,basic,bata,batas,batasang,batid,bayan,bayanihan,begun,belief,belong,beloved,beneficial,beneficiaries,benefit,benefiting,benefits,benipisyaryo,beses,beset,bibilhin,bid,biktima,blood,blueprint,bonds,boost,booster,bot,bottleneck,bottlenecks,brand,branded,breadth,breakthrough,breed,bring,broad,broadband,brought,brown,budget,budgeted,budgeting,buhay,build,building,buildings,bukas,bukod,buksan,buksang,bulto,bumaba,bumalik,burahin,burden,bureau,bureaucracy,bureaus,bus,business,buwan,cabinet,calculated,called,capacity,capita,capital,capitalizing,carbon,care,careful,cash,catch,category,caused,cavite,cbs,cdc,cebu,center,centers,certifications,chain,chains,challenges,change,channel,characterized,charles,cheap,cheapest,cheers,chief,child,children,cities,citizen,citizens,city,civil,clarification,clarifying,class,classes,classrooms,climate,clinic,close,clouds,coherence,coinciding,cold,collaborate,collaboration,collection,college,collusion,combination,combined,command,commerce,commercial,commission,commit,commitment,committed,commodities,common,communicated,communications,communities,community,commuter,companies,compared,compete,competent,competition,competitive,complementary,complete,completed,complex,compliance,complications,comply,component,composed,comprehensive,computers,computing,concerned,concurrence,concurrent,condition,conditions,condonation,condoned,conduct,confidently,conflict,confronted,congress,connect,connecting,connectivity,consensus,consideration,considered,consistent,consolidation,consumer,consumption,context,continuation,continue,continued,continues,continuous,continuously,contracts,contributor,control,convenient,conventional,converting,cooperate,cooperation,cooperatives,coordinate,coordination,cornerstones,coronavirus,corp,corporate,corporation,corps,cost,counselling,countries,country,countrymen,courses,court,cover,coverage,covid,create,created,creates,creation,creative,creativity,credible,crisis,critical,crucial,crude,cultural,culture,current,custodians,customs,cut,daan,dagat,dagdag,dala,dalawang,dark,data,database,daunting,davao,day,dayuhang,deal,debt,decision,declarations,decongest,deeply,defense,deficit,degrees,delivered,delivery,demand,demonstrated,denr,department,departments,depensa,deploy,deployment,depth,derivations,deserve,desired,destination,determine,determining,develop,developed,developing,development,developments,devices,dict,difficult,digit,digital,digitalization,digitized,dilg,diminish,dinadagdagan,diplomatic,direct,directed,direst,disabilities,disability,disadvantaged,disaster,disbursement,disbursements,discipline,discussion,discussions,disease,disrupted,distinction,distinguished,distressed,distribution,diversifying,divide,doctors,document,doh,doktor,dollar,domestic,doors,dost

In [41]:
marcos_idf = marcos_idf.stack().reset_index()
marcos_idf

,level_0,level_1,0
0,0,sona_no,0
1,0,term,aagapay
2,0,tfidf,0.007936
3,1,sona_no,0
4,1,term,aalaga
...,...,...,...
5863,1954,term,zimmerman
5864,1954,tfidf,0.007936
5865,1955,sona_no,0
5866,1955,term,zubiri


In [28]:
marcos_idf = marcos_idf.rename(columns={'level_0': 'sona_no','level_1': 'term', 'tfidf': 'term', 0: 'tfidf'})
marcos_idf.head()

,sona_no,term,tfidf
0,0,aagapay,0.007936
1,0,aalaga,0.007936
2,0,aalis,0.007936
3,0,aaral,0.007936
4,0,aatasan,0.007936


In [29]:
marcos_firstsona = marcos_idf.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
marcos_firstsona.head()

,sona_no,term,tfidf
1199,0,natin,0.253960
666,0,government,0.198406
397,0,department,0.174598
1202,0,national,0.174598
1369,0,philippines,0.166661


## Most relevant words from Marcos Jr.'s first SONA

In [32]:
# adding a little randomness to break ties in term ranking
marcos_firstsona_plusRand = marcos_firstsona.copy()
marcos_firstsona_plusRand['tfidf'] = marcos_firstsona_plusRand['tfidf'] + np.random.rand(marcos_firstsona.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(marcos_firstsona_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + text).properties(width = 600, height=400)

alt.LayerChart(...)

## Diosdado Macapagal

In here, we are testing speeches made by one pre-martial law president, **Diosdado Macapagal**.

In [38]:
X2 = vectorizer.fit_transform(macapagal['speech'])
X2

<4x4497 sparse matrix of type '<class 'numpy.float64'>'
	with 6973 stored elements in Compressed Sparse Row format>

In [40]:
macapagal_vectors = pd.DataFrame(X2.toarray(), columns=vectorizer.get_feature_names_out())
# [print(x) for x in macapagal.speech]
macapagal_vectors.round(2)

abatement  abide  abides  abiding  abilities  ability  abolish  abolished  \
0       0.00   0.00    0.00     0.00       0.00     0.00     0.00       0.00   
1       0.01   0.01    0.01     0.00       0.00     0.01     0.01       0.01   
2       0.00   0.00    0.00     0.00       0.06     0.05     0.02       0.00   
3       0.00   0.00    0.00     0.01       0.00     0.01     0.00       0.00   

   abolition  absence  absentee  abso  absolute  absorb  absorbed  absorbing  \
0       0.00     0.01      0.00  0.00      0.00    0.03      0.02       0.00   
1       0.01     0.00      0.01  0.01      0.00    0.00      0.00       0.00   
2       0.00     0.00      0.00  0.00      0.00    0.00      0.00       0.00   
3       0.00     0.00      0.00  0.00      0.01    0.00      0.00       0.01   

   abundance  abuse  abyss   aca  accelerate  accelerated  accelerating  \
0       0.01   0.00   0.01  0.00        0.01         0.00          0.00   
1       0.00   0.01   0.00  0.00        0.00         0.00          0.00   
2       0.00   0.00   0.00  0.00        0.00         0.00          0.00   
3       0.00   0.00   0.00  0.01        0.02         0.03          0.01   

   accentuated  accept  acceptable  acceptance  accepted  accepting  \
0         0.00    0.00        0.00        0.00      0.00       0.00   
1         0.00    0.02        0.01        0.01      0.01       0.01   
2         0.00    0.00        0.00        0.00      0.04       0.00   
3         0.01    0.01        0.00        0.01      0.00       0.00   

   accessible  acclaimed  acclimated  accommoda  accommodate  accompanied  \
0        0.00       0.00        0.00       0.01         0.00         0.00   
1        0.00       0.00        0.01       0.00         0.00         0.00   
2        0.00       0.03        0.00       0.00         0.00         0.00   
3        0.01       0.00        0.00       0.00         0.01         0.01   

   accompany  accomplish  accomplished  accomplishment  accomplishments  \
0       0.00        0.00          0.00            0.01             0.00   
1       0.01        0.01          0.00            0.01             0.00   
2       0.00        0.01          0.02            0.00             0.00   
3       0.00        0.00          0.01            0.00             0.01   

   accord  accorded  account  accounting  accounts  accrue  accruing  achieve  \
0    0.00      0.00     0.00        0.00      0.00    0.00      0.00     0.03   
1    0.01      0.01     0.00        0.01      0.01    0.01      0.01     0.01   
2    0.00      0.00     0.00        0.00      0.00    0.00      0.00     0.07   
3    0.00      0.00     0.01        0.00      0.00    0.00      0.00     0.02   

   achieved  achievement  achievements  achieving  achilles  acknowledgment  \
0      0.01         0.00          0.00       0.01      0.00            0.00   
1      0.03         0.00          0.01       0.00      0.00            0.01   
2      0.03         0.05          0.00       0.00      0.00            0.00   
3      0.02         0.02          0.00       0.01      0.01            0.00   

   acquire  acquiring  acquisition  acted  acting  action  actions  active  \
0     0.00       0.00         0.00   0.00    0.01    0.02     0.01    0.00   
1     0.01       0.00         0.01   0.01    0.01    0.07     0.02    0.01   
2     0.00       0.00         0.00   0.00    0.00    0.00     0.00    0.05   
3     0.00       0.01         0.01   0.01    0.00    0.02     0.00    0.01   

   actively  activities  activity  acts  actual  actuality  acutely   add  \
0      0.01        0.02      0.02  0.01    0.00       0.00     0.00  0.01   
1      0.01        0.02      0.01  0.00    0.01       0.00     0.00  0.00   
2      0.00        0.00      0.00  0.02    0.00       0.00     0.00  0.00   
3      0.00        0.07      0.01  0.00    0.02       0.01     0.01  0.01   

   adding  addition  additional  additions  address  addressed   ade  \
0    0.00      0.00        0.02       0.00     0.01     

In [42]:
macapagal_vectors = macapagal_vectors.stack().reset_index()
macapagal_vectors

,level_0,level_1,0
0,0,abatement,0.000000
1,0,abide,0.000000
2,0,abides,0.000000
3,0,abiding,0.000000
4,0,abilities,0.000000
...,...,...,...
17983,3,yielding,0.000000
17984,3,zambales,0.022184
17985,3,zamboanga,0.022184
17986,3,zation,0.000000


In [43]:
macapagal_vectors = macapagal_vectors.rename(columns={'level_0': 'sona_no','level_1': 'term', 'tfidf': 'term', 0: 'tfidf'})
macapagal_vectors.head()

,sona_no,term,tfidf
0,0,abatement,0.0
1,0,abide,0.0
2,0,abides,0.0
3,0,abiding,0.0
4,0,abilities,0.0


In [44]:
macapagal_sona = macapagal_vectors.sort_values(by=['sona_no','tfidf'], ascending=[True,False]).groupby(['sona_no']).head(10)
macapagal_sona.head()

,sona_no,term,tfidf
1330,0,economic,0.343128
3201,0,program,0.290706
1858,0,government,0.238283
2950,0,people,0.204924
1746,0,foreign,0.190627


## Most relevant words in Macapagal SONAs

In [45]:
# adding a little randomness to break ties in term ranking
macapagal_sona_plusRand = macapagal_sona.copy()
macapagal_sona_plusRand['tfidf'] = macapagal_sona_plusRand['tfidf'] + np.random.rand(macapagal_sona.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(macapagal_sona_plusRand).encode(
    x = 'rank:O',
    y = 'sona_no:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["sona_no"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + text).properties(width = 600, height=400)

alt.LayerChart(...)